# Outlook

In this notebook, using BBRL, we code a version of the DQN algorithm with a replay buffer and a target network. To understand this code, you need to know more about BBRL. You should first have a look at [the BBRL interaction model](https://colab.research.google.com/drive/1_yp-JKkxh_P8Yhctulqm0IrLbE41oK1p?usp=sharing), then [a first example](https://colab.research.google.com/drive/1Ui481r47fNHCQsQfKwdoNEVrEiqAEokh?usp=sharing) and, most importantly, [details about the data collection implementation](https://colab.research.google.com/drive/1W9Y-3fa6LsPeR6cBC1vgwBjKfgMwZvP5?usp=sharing).

The DQN algorithm is explained in [this video](https://www.youtube.com/watch?v=CXwvOMJujZk) and you can also read [the corresponding slides](http://pages.isir.upmc.fr/~sigaud/teach/dqn.pdf).

In this notebook, you will learn how to modify the previous notebook:

- to use a replay buffer and an environment that resets
- to use a target network for $Q$
- to use a better estimation for the maximum (Double-DQN)

## Installation and Imports

### Installation

The BBRL library is [here](https://github.com/osigaud/bbrl).

This is OmegaConf that makes it possible that by just defining the `def run_dqn(cfg):` function and then executing a long `params = {...}` variable at the bottom of this colab, the code is run with the parameters without calling an explicit main.

More precisely, the code is run by calling

`config=OmegaConf.create(params)`

`run_dqn(config)`

at the very bottom of the colab, after starting tensorboard.

In [ ]:

try:
    from easypip import easyimport
except:
    !pip install easypip
    from easypip import easyimport

In [ ]:
import os
import functools
import time
from typing import Tuple

OmegaConf = easyimport("omegaconf").OmegaConf
torch = easyimport("torch")
bbrl_gym = easyimport("bbrl_gym")
import gym
import bbrl
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import copy

### BBRL imports

In [ ]:
from bbrl.agents.agent import Agent
from bbrl import get_arguments, get_class, instantiate_class

# The workspace is the main class in BBRL, this is where all data is collected and stored
from bbrl.workspace import Workspace

# Agents(agent1,agent2,agent3,...) executes the different agents the one after the other
# TemporalAgent(agent) executes an agent over multiple timesteps in the workspace, 
# or until a given condition is reached
from bbrl.agents import Agents, RemoteAgent, TemporalAgent

# AutoResetGymAgent is an agent able to execute a batch of gym environments
# with auto-resetting. These agents produce multiple variables in the workspace: 
# ’env/env_obs’, ’env/reward’, ’env/timestep’, ’env/done’, ’env/initial_state’, ’env/cumulated_reward’, 
# ... When called at timestep t=0, then the environments are automatically reset. 
# At timestep t>0, these agents will read the ’action’ variable in the workspace at time t − 1
from bbrl.agents.gymb import AutoResetGymAgent, NoAutoResetGymAgent
# Not present in the A2C version...
from bbrl.utils.logger import TFLogger
from bbrl.utils.replay_buffer import ReplayBuffer

def make_env(env_name):
    return gym.make(env_name)

## The replay buffer

Differently from the previous case, we use a replace buffer that stores the a set of transitions $(s_t, a_t, r_t, s_{t+1})$

In [ ]:
# We deal with 3 running episodes at a time (random seed 2139)
env_agent = NoAutoResetGymAgent(make_env, {'env_name': 'CartPole-v1'}, 1, 2139)
obs_size, action_dim = env_agent.get_obs_and_actions_sizes()
print(f"Environment: observation space in R^{obs_size} and action space R^{action_dim}")

class RandomAgent(Agent):
    def __init__(self, action_dim):
        super().__init__()
        self.action_dim = action_dim

    def forward(self, t: int, choose_action=True, **kwargs):
        """An Agent can use self.workspace"""
        obs = self.get(("env/env_obs", t))
        action = torch.randint(0, self.action_dim, (len(obs), ))
        self.set(("action", t), action)

# Each agent will be run (in the order given when constructing Agents)
agents = Agents(env_agent, RandomAgent(action_dim))
t_agents = TemporalAgent(agents)

In [ ]:
# Creates a new workspace
workspace = Workspace() 
t_agents(workspace, stop_variable="env/done")

# We get the transitions: each tensor is transformed so
# that: 
# - we have the value at time step t and t+1 (so all the tensors first dimension have a size of 2)
# - there is no distinction between the different environments (here, there is just one environment run in parallel to make it easy)
transitions = workspace.get_transitions()

# You can see that each pair of actions in the transitions can be found in the workspace
display("Observations (first 3)", workspace["env/env_obs"][:3, 0])

display("Transitions of actions (first 3)")
for t in range(3):
    display(f'(s_{t}, s_{t+1})')
    display(transitions["env/env_obs"][:, t])

In [ ]:
# Finally, the replay buffer keeps slices [:, i, ...] of the transition workspace
# (here at most 100 transitions)
rb = ReplayBuffer(max_size=100)

# We add the transitions to the buffer....
rb.put(transitions)

# And sample from them
# here we get 3 tuples (s_t, s_{t+1})
rb.get_shuffled(3)["env/env_obs"]

A transition workspace is still a workspace... this is quite handy since each transition can be seen as a mini-episode of two time steps; we can use our agents on it:

In [ ]:
# Just as a reference

display(transitions["action"])

t_random_agent = TemporalAgent(RandomAgent(action_dim))
t_random_agent(transitions, t=0, n_steps=2)

# Here, the action tensor will have been overwritten by the new actions
display(transitions["action"])

## Definition of agents

The [DQN](https://daiwk.github.io/assets/dqn.pdf) algorithm is a critic only algorithm. Thus we just need a Critic agent (which will also be used to output actions) and an Environment agent. We reuse the `DiscreteQAgent` class that we have already explained in the previous notebook.

In [ ]:
def build_mlp(sizes, activation, output_activation=nn.Identity()):
    layers = []
    for j in range(len(sizes) - 1):
        act = activation if j < len(sizes) - 2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j + 1]), act]
    return nn.Sequential(*layers)

In [ ]:
class DiscreteQAgent(Agent):
    def __init__(self, state_dim, hidden_layers, action_dim):
        super().__init__()
        self.model = build_mlp(
            [state_dim] + list(hidden_layers) + [action_dim], activation=nn.ReLU()
        )

    def forward(self, t, choose_action=True, **kwargs):
        obs = self.get(("env/env_obs", t))
        q_values = self.model(obs)
        self.set(("q_values", t), q_values)

        # Sets the action
        if choose_action:
            action = q_values.argmax(1)
            self.set(("action", t), action)

### Creating an Exploration method

As Q-learning, DQN needs some exploration to prevent too early convergence. Here we will use the simple $\epsilon$-greedy exploration method. The method is implemented as an agent which chooses an action based on the Q-values.

In [ ]:
class EGreedyActionSelector(Agent):
    def __init__(self, epsilon):
        super().__init__()
        self.epsilon = epsilon

    def forward(self, t, **kwargs):
        q_values = self.get(("q_values", t))
        nb_actions = q_values.size()[1]
        size = q_values.size()[0]
        is_random = torch.rand(size).lt(self.epsilon).float()
        random_action = torch.randint(low=0, high=nb_actions, size=(size,))
        max_action = q_values.max(1)[1]
        action = is_random * random_action + (1 - is_random) * max_action
        action = action.long()
        self.set(("action", t), action)

### Training and evaluation environments

We build two environments: one for training and another one for evaluation.

For training, it is more efficient to use an AutoResetGymAgent, as we do not want to waste time if the task is done in an environment sooner than in the others.

By contrast, for evaluation, we just need to perform a fixed number of episodes (for statistics), thus it is more convenient to use a NoAutoResetGymAgent with a set of environments and just run one episode in each environment. Thus we can use the `env/done` stop variable and take the average over the cumulated reward of all environments.


See [this notebook](https://colab.research.google.com/drive/1Ui481r47fNHCQsQfKwdoNEVrEiqAEokh?usp=sharing) for explanations about agents and environment agents.

In [ ]:
def get_env_agents(cfg) -> Tuple[AutoResetGymAgent, NoAutoResetGymAgent]:
    # Train environment
    train_env_agent = AutoResetGymAgent(
        get_class(cfg.gym_env),
        get_arguments(cfg.gym_env),
        cfg.algorithm.n_envs,
        cfg.algorithm.seed,
    )

    # Evaluation environment
    eval_env_agent = NoAutoResetGymAgent(
    get_class(cfg.gym_env),
    get_arguments(cfg.gym_env),
    cfg.algorithm.nb_evals,
    cfg.algorithm.seed,
    )
    return train_env_agent, eval_env_agent

### Create the DQN agent

Interestingly, the loop between the policy and the environment is first defined as a collection of agents, and then embedded into a single TemporalAgent.

In [ ]:
def create_dqn_agent(cfg, train_env_agent, eval_env_agent):
    obs_size, act_size = train_env_agent.get_obs_and_actions_sizes()

    # Get the two agents (critic and target critic)
    critic = DiscreteQAgent(obs_size, cfg.algorithm.architecture.hidden_size, act_size)
    target_critic = copy.deepcopy(critic)

    # Builds the train agent that will produce transitions
    explorer = EGreedyActionSelector(cfg.algorithm.epsilon)
    tr_agent = Agents(train_env_agent, critic, explorer)
    train_agent = TemporalAgent(tr_agent)

    # Creates two temporal agents just for "replaying" some parts
    # of the transition buffer    
    q_agent = TemporalAgent(critic)
    target_q_agent = TemporalAgent(target_critic)


    # Get an agent that is executed on a complete workspace
    ev_agent = Agents(eval_env_agent, critic)
    eval_agent = TemporalAgent(ev_agent)
    train_agent.seed(cfg.algorithm.seed)

    return train_agent, eval_agent, q_agent, target_q_agent

### The Logger class

Explanations for the logger were already given in [this notebook](https://colab.research.google.com/drive/1raeuB6uUVUpl-4PLArtiAoGnXj0sGjSV?usp=sharing).

In [ ]:
class Logger():

  def __init__(self, cfg):
    self.logger = instantiate_class(cfg.logger)

  def add_log(self, log_string, loss, epoch):
    self.logger.add_scalar(log_string, loss.item(), epoch)

  # Log losses
  def log_losses(self, cfg, epoch, critic_loss, entropy_loss, actor_loss):
    self.add_log("critic_loss", critic_loss, epoch)
    self.add_log("entropy_loss", entropy_loss, epoch)
    self.add_log("actor_loss", actor_loss, epoch)



### Setup the optimizers

We use a single optimizer to tune the parameters of the actor (in the prob_agent part) and the critic (in the critic_agent part). It would be possible to have two optimizers which would work separately on the parameters of each component agent, but it would be more complicated because updating the actor requires the gradient of the critic.

In [ ]:
# Configure the optimizer over the q agent
def setup_optimizers(cfg, q_agent):
    optimizer_args = get_arguments(cfg.optimizer)
    parameters = q_agent.parameters()
    optimizer = get_class(cfg.optimizer)(parameters, **optimizer_args)
    return optimizer

### Compute critic loss

Detailed explanations of the function to compute the critic loss when using a NoAutoResetGymAgent are given in [this notebook](https://colab.research.google.com/drive/1raeuB6uUVUpl-4PLArtiAoGnXj0sGjSV).

The case where we use the AutoResetGymAgent is very similar, but we need to specify that we use the first part of the Q-values (`q_values[0]`) for representing $Q(s_t,a_t)$ and the second part (`q_values[1]`) for representing $Q(s_{t+1},a)$, as these values are stored into a transition model.

In [ ]:
def compute_critic_loss(cfg, reward, must_bootstrap, q_values, target_q_values, action):
    # À compléter...  
    assert False, 'Code non implémenté'
    #Don't forget that we deal with transitions (and not episodes)

    # Compute critic loss (no need to use must_bootstrap here since we are dealing with "full" transitions)
    mse = nn.MSELoss()
    critic_loss = mse(target, qvals)
    return critic_loss

## Main training loop

Note that everything about the shared workspace between all the agents is completely hidden under the hood. This results in a gain of productivity, at the expense of having to dig into the BBRL code if you want to understand the details, change the multiprocessing model, etc.

### Agent execution

This is the tricky part with BBRL, the one we need to understand in detail. The difficulty lies in the copy of the last step and the way to deal with the n_steps return.

The call to `train_agent(workspace, t=1, n_steps=cfg.algorithm.n_timesteps - 1, stochastic=True)` makes the agent run a number of steps in the workspace. In practice, it calls the [`__call__(...)`](https://github.com/osigaud/bbrl/blob/master/bbrl/agents/agent.py#L54) function which makes a forward pass of the agent network using the workspace data and updates the workspace accordingly.

Now, if we start at the first epoch (`epoch=0`), we start from the first step (`t=0`). But when subsequently we perform the next epochs (`epoch>0`), we must not forget to cover the transition at the border between the previous epoch and the current epoch. To avoid this risk, we copy the information from the last time step of the previous epoch into the first time step of the next epoch. This is explained in more details in [a previous colab](https://colab.research.google.com/drive/1W9Y-3fa6LsPeR6cBC1vgwBjKfgMwZvP5).

The [previous notebook](https://colab.research.google.com/drive/1W9Y-3fa6LsPeR6cBC1vgwBjKfgMwZvP5) explains a lot of these details. In particular, read it to understand the `execute_agents(...)` function, the `transition_workspace = train_workspace.get_transitions()` line and the computation of `must_bootstrap`.

Note that we `optimizer.zero_grad()`, `loss.backward()` and `optimizer.step()` lines. `optimizer.zero_grad()` is necessary to cancel all the gradients computed at the previous iterations

In [ ]:
def run_dqn(cfg, compute_critic_loss):
    # 1)  Build the  logger
    logger = Logger(cfg)
    best_reward = -10e9

    # 2) Create the environment agent
    train_env_agent, eval_env_agent = get_env_agents(cfg)

    # 3) Create the DQN-like Agent
    train_agent, eval_agent, q_agent, target_q_agent = create_dqn_agent(
        cfg, train_env_agent, eval_env_agent
    )

    # 5) Configure the workspace to the right dimension
    # Note that no parameter is needed to create the workspace.
    # In the training loop, calling the agent() and critic_agent()
    # will take the workspace as parameter
    train_workspace = Workspace()  # Used for training
    rb = ReplayBuffer(max_size=cfg.algorithm.buffer_size)

    # 6) Configure the optimizer over the a2c agent
    optimizer = setup_optimizers(cfg, q_agent)
    nb_steps = 0
    last_eval_step = 0
    last_critic_update_step = 0

    # 7) Training loop
    for epoch in range(cfg.algorithm.max_epochs):
        # Execute the agent in the workspace
        if epoch > 0:
            train_workspace.zero_grad()
            train_workspace.copy_n_last_steps(1)
            train_agent(
                train_workspace, t=1, n_steps=cfg.algorithm.n_steps - 1, stochastic=True
            )
        else:
            train_agent(
                train_workspace, t=0, n_steps=cfg.algorithm.n_steps, stochastic=True
            )

        # Get the transitions
        transition_workspace = train_workspace.get_transitions()

        action = transition_workspace["action"]
        nb_steps += action[0].shape[0]
        
        # Adds the transitions to the workspace
        rb.put(transition_workspace)
        for _ in range(cfg.algorithm.n_updates):
            rb_workspace = rb.get_shuffled(cfg.algorithm.batch_size)

            # The q agent needs to be executed on the rb_workspace workspace (gradients are removed in workspace)
            q_agent(rb_workspace, t=0, n_steps=2, choose_action=False)
            q_values, done, truncated, reward, action = rb_workspace[
                "q_values", "env/done", "env/truncated", "env/reward", "action"
            ]

            with torch.no_grad():
                target_q_agent(rb_workspace, t=0, n_steps=2, stochastic=True)
            target_q_values = rb_workspace["q_values"]

            # Determines whether values of the critic should be propagated
            # True if the episode reached a time limit or if the task was not done
            # See https://colab.research.google.com/drive/1erLbRKvdkdDy0Zn1X_JhC01s1QAt4BBj
            must_bootstrap = torch.logical_or(~done[1], truncated[1])

            if rb.size() > cfg.algorithm.learning_starts:
              # Compute critic loss
              critic_loss = compute_critic_loss(
                  cfg, reward, must_bootstrap, q_values, target_q_values[1], action
              )
              # Store the loss for tensorboard display
              logger.add_log("critic_loss", critic_loss, nb_steps)

              optimizer.zero_grad()
              critic_loss.backward()
              torch.nn.utils.clip_grad_norm_(q_agent.parameters(), cfg.algorithm.max_grad_norm)
              optimizer.step()
              if nb_steps - last_critic_update_step > cfg.algorithm.target_critic_update:
                  last_critic_update_step = nb_steps
                  target_q_agent.agent = copy.deepcopy(q_agent.agent)

        if nb_steps - last_eval_step > cfg.algorithm.eval_interval:
            last_eval_step = nb_steps
            eval_workspace = Workspace()  # Used for evaluation
            eval_agent(
                eval_workspace, t=0, stop_variable="env/done", choose_action=True
            )
            rewards = eval_workspace["env/cumulated_reward"][-1]
            mean = rewards.mean()
            logger.add_log("reward", mean, nb_steps)
            print(f"epoch: {epoch}, reward: {mean}")
            if cfg.save_best and mean > best_reward:
                best_reward = mean
                directory = "./dqn_critic/"
                if not os.path.exists(directory):
                    os.makedirs(directory)
                filename = directory + "dqn0_" + str(mean.item()) + ".agt"
                eval_agent.save_model(filename)



## Definition of the parameters

The logger is defined as `bbrl.utils.logger.TFLogger` so as to use a tensorboard visualisation.

### Launching tensorboard to visualize the results

In [ ]:
# For Colab - otherwise, it is easier and better to launch tensorboard from
# the terminal
if get_ipython().__class__.__module__ == "google.colab._shell":
    pass
    %load_ext tensorboard
    %tensorboard --logdir ./tmp
else:
    import sys
    import os
    import os.path as osp
    print(f"Launch tensorboard from the shell:\n{osp.dirname(sys.executable)}/tensorboard --logdir={os.getcwd()}/tmp")

In [ ]:
params={
  "save_best": False,
  "logger":{
    "classname": "bbrl.utils.logger.TFLogger",
    "log_dir": "./tmp/dqn-buffer-" + str(time.time()),
    "cache_size": 10000,
    "every_n_seconds": 10,
    "verbose": False,    
    },

  "algorithm":{
    "seed": 4,
    "max_grad_norm": 0.5,
    "epsilon": 0.02,
    "n_envs": 8,
    "n_steps": 32,
    "n_updates": 32,
    "eval_interval": 2000,
    "learning_starts": 2000,
    "nb_evals": 10,
    "buffer_size": 1e6,
    "batch_size": 256,
    "target_critic_update": 5000,
    "max_epochs": 3500,
    "discount_factor": 0.99,
    "architecture":{"hidden_size": [128, 128]},
  },
  "gym_env":{
    "classname": "__main__.make_env",
    "env_name": "CartPole-v1",
  },
  "optimizer":
  {
    "classname": "torch.optim.Adam",
    "lr": 1e-3,
  }
}

config=OmegaConf.create(params)
torch.manual_seed(config.algorithm.seed)
run_dqn(config, compute_critic_loss)

The version used in this colab uses $< s_t, a_t, r_t, s_{t+1}>$ samples. As an exercise, you may switch to $< s_t, a_t, r_{t+1}, s_{t+1}>$ samples, going back to the standard SaLinA notation. For that, replace the import to `bbrl.agents.gyma` instead of `gymb`, and change the temporal difference update rule (in `compute_critic_loss(...)`) accordingly. See [this notebook](https://colab.research.google.com/drive/1Cld72_FBA1aMS2U4EsyV3LGZIlQC_PsC?usp=sharing) for more explanations.

## Coding Exercise: Double DQN (DDQN)

In DQN, the same network is responsible for selecting and estimating the best next action (in the TD-target) and that may lead to over-estimation: the action which q-value is over-estimated will be chosen more often. As a result, training is slower.

To reduce over-estimation, double q-learning (and then DDQN) was proposed. It decouples the action selection from the value estimation.

Concretely, in DQN, the target value in the critic loss (used to update the Q critic) for a sample at time $t$ is defined as:

$$Y^{DQN}_{t} = r_{t+1} + \gamma{Q}\left(s_{t+1}, \arg\max_{a}Q\left(s_{t+1}, a; \mathbb{\theta}_{target}\right); \mathbb{\theta}_{target}\right)$$

where the target network `target_q_agent` with parameters $\mathbb{\theta}_{target}$ is used for both action selection and estimation, and can therefore be rewritten:

$$Y^{DQN}_{t} = r_{t+1} + \gamma \max_{a}{Q}\left(s_{t+1}, a; \mathbb{\theta}_{target}\right)$$

Instead, DDQN uses the online critic `q_agent` with parameters $\mathbb{\theta}_{online}$ to select the action, whereas it uses the target network `target_q_agent` to estimate the associated Q-values:

$$Y^{DDQN}_{t} = r_{t+1} + \gamma{Q}\left(s_{t+1}, \arg\max_{a}Q\left(s_{t+1}, a; \mathbb{\theta}_{online}\right); \mathbb{\theta}_{target}\right)$$


The goal in this exercise is for you to write the update method for `DDQN`.

In [ ]:
def compute_ddqn_loss(cfg, reward, must_bootstrap, q_values, target_q_values, action):
    # À compléter...  
    assert False, 'Code non implémenté'

    # Compute critic loss
    mse = nn.MSELoss()
    critic_loss = mse(target, qvals)
    return critic_loss

In [ ]:
params={
  "save_best": False,
  "logger":{
    "classname": "bbrl.utils.logger.TFLogger",
    "log_dir": "./tmp/ddqn-buffer-" + str(time.time()),
    "cache_size": 10000,
    "every_n_seconds": 10,
    "verbose": False,    
    },

  "algorithm":{
    "seed": 4,
    "max_grad_norm": 0.5,
    "epsilon": 0.02,
    "n_envs": 8,
    "n_steps": 32,
    "eval_interval": 2000,
    "learning_starts": 2000,
    "nb_evals": 10,
    "buffer_size": 1e6,
    "batch_size": 256,
    "target_critic_update": 5000,
    "max_epochs": 3500,
    "discount_factor": 0.99,
    "architecture":{"hidden_size": [128, 128]},
  },
  "gym_env":{
    "classname": "__main__.make_env",
    "env_name": "CartPole-v1",
  },
  "optimizer":
  {
    "classname": "torch.optim.Adam",
    "lr": 1e-3,
  }
}

config=OmegaConf.create(params)
torch.manual_seed(config.algorithm.seed)
run_dqn(config, compute_ddqn_loss)